# Llama 2 Code

### Run the following cells to install required packages


In [ ]:
!pip install sentencepiece==0.1.99
!pip install transformers==4.31.0
!pip install accelerate==0.21.0
!pip install bitsandbytes==0.41.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00


### Loading Llama 2




In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

hf_access_token = "INSERT YOUR TOKEN HERE"

tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=hf_access_token)
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", load_in_8bit=True, device_map="auto", token=hf_access_token)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Llama2 Prompting with Special Tokens



$\quad$ \- According to the instructions mentioned in: https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L212


### You can initialize the generation configuration here:

*   temperature
*   top_p
*   repetition_penalty



In [ ]:
def llama2_talk(text):
    gen_len = 512
    generation_kwargs = {
          "max_new_tokens": gen_len,
          "top_p": 0.9,
          "temperature": 0.6,
          "repetition_penalty": 1.2,
          "do_sample": True,
      }

    B_INST, E_INST = "[INST]", "[/INST]"

    prompt_text = text

    prompt = f"{B_INST} {prompt_text} {E_INST}"  # Special format reuired by the Llama2 Chat Model

    prompt_ids = tokenizer(prompt, return_tensors="pt")

    prompt_size = prompt_ids['input_ids'].size()[1]

    generate_ids = model.generate(prompt_ids.input_ids.to(model.device), **generation_kwargs)

    generate_ids = generate_ids.squeeze()

    response = tokenizer.decode(generate_ids.squeeze()[prompt_size+1:], skip_special_tokens=True).strip()

    print("Llama response: ", response)

### Code with System Prompts (Optional)



*   System Prompts Provide more context about the underlying task to the model: https://huggingface.co/blog/llama2

In [ ]:
def llama2_system_talk(system, text):
    gen_len = 512

    generation_kwargs = {
          "max_new_tokens": gen_len,
          "top_p": 0.9,
          "temperature": 0.6,
          "repetition_penalty": 1.2,
          "do_sample": True,
      }

    B_INST, E_INST = "[INST]", "[/INST]"

    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    system_prompt = system

    prompt_text = text

    prompt = f"{B_INST} {B_SYS} {system_prompt} {E_SYS} {prompt_text} {E_INST}"  # Special format required by the Llama2 Chat Model where we can use system messages to provide more context about the task

    prompt_ids = tokenizer(prompt, return_tensors="pt")

    prompt_size = prompt_ids['input_ids'].size()[1]

    generate_ids = model.generate(prompt_ids.input_ids.to(model.device), **generation_kwargs)

    generate_ids = generate_ids.squeeze()

    response = tokenizer.decode(generate_ids.squeeze()[prompt_size+1:], skip_special_tokens=True).strip()

    print("Llama response: ", response)

#Examples:

In [ ]:
input_text = "The movie Openhimer is the best movie ever to come out in 2023 \n What is the sentiment of the following sentence? Is it positive, neutral or negative?"

llama2_talk(input_text)

Llama response:  Based on the sentence "The movie Openhimer is the best movie ever to come out in 2023," the sentiment is POSITIVE. The use of the word "best" and the exclamation point convey enthusiasm and excitement about the movie, indicating a positive sentiment.


#Examples with System Prompts (Optional):

In [ ]:
system_prompt = "You are a helpful, respectful and honest assistant. Your role is to read the given text and identify the corresponding sentiment"

input_text = "The movie Openhimer is the best movie ever to come out in 2023 \n What is the sentiment of the following sentence? Is it positive, neutral or negative?"

llama2_system_talk(system_prompt, input_text)

Llama response:  Great! Based on the information provided, the sentiment of the sentence "The movie Openhimer is the best movie ever to come out in 2023" is POSITIVE.
